In [2]:
import pandas as pd
from geopy.geocoders import Photon
from geopy.geocoders import Nominatim
from unidecode import unidecode
import unidecode

In [3]:
import numpy as np
from shapely.geometry import Point, Polygon

In [4]:
import geopandas as gpd

### Seleccionar las bd que conformaran nuestra bd de presencia institucional de Sociedad Civil


-Directorio OIM


In [ ]:
directorio_full = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/SC/seleccionado de instituciones/institucionesmigracionfull.csv')

In [ ]:
directorio_full = directorio_full.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])

# Resetear el índice y descartar el índice anterior
directorio_full.reset_index(drop=True, inplace=True)

In [ ]:
# Verificar cambios
directorio_full.head(50)

In [ ]:
# Paso 1: Eliminar espacios al inicio en 'name'
directorio_full['name'] = directorio_full['name'].str.lstrip()

# Paso 2: Quitar acentos
def quitar_acentos(texto):
    if isinstance(texto, str):
        return unidecode.unidecode(texto)
    return texto

for columna in directorio_full.select_dtypes(include=['object']).columns:
    directorio_full[columna] = directorio_full[columna].apply(quitar_acentos)

In [ ]:
directorio_full['bd'].unique()

### separamos en diferentes bds

In [ ]:
Directorio_OIM = directorio_full[directorio_full['bd'] == 'Directorio OIM']


In [ ]:
# Lista de valores a reemplazar
valores_a_reemplazar = ['Centros de Asistencia Social', 'Centro de Asistencia Social']

# Reemplazar los valores en la columna 'descripcion'
Directorio_OIM['descripcion'] = Directorio_OIM['descripcion'].replace(valores_a_reemplazar, 'Centros de Atencion Social')

# Verificar los valores únicos después del reemplazo
print(Directorio_OIM['descripcion'].unique())

In [ ]:
Directorio_OIM['descripcion'].unique()

In [ ]:
Directorio_OIM.shape

In [ ]:
ACNUR_albergues = directorio_full[directorio_full['bd'] == 'ACNUR albergues']
ACNUR_servicios = directorio_full[directorio_full['bd'] == 'ACNUR servicios']

### directorios seleccionados:

In [ ]:
CAS = Directorio_OIM[Directorio_OIM['descripcion'] == 'Centros de Atencion Social']

In [ ]:
grupos_beta = Directorio_OIM[Directorio_OIM['descripcion'] == 'Grupos Beta']

In [ ]:
salud = Directorio_OIM[Directorio_OIM['descripcion'] == 'Salud']

In [ ]:
import pandas as pd

# Concatenar DataFrames
resultante_df = pd.concat([grupos_beta, salud], ignore_index=True)


In [ ]:
# Guardar el DataFrame resultante en un archivo .xlsx
ruta_archivo = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/SC/seleccionado de instituciones/cruzrojagruposbeta.xlsx'  # Cambia esto por tu ruta y nombre de archivo real

resultante_df.to_excel(ruta_archivo, index=False, engine='openpyxl')


# dataframe completo

Incluye centros de atención social extraidos de los directorios de OIM, Redes, ACNUR.

In [ ]:
df1 = pd.read_excel('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/SC/seleccionado de instituciones/instituciones_sc.xlsx')

In [ ]:
grid = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/grid.csv')


In [ ]:
from shapely.wkt import loads
if 'geometry' in grid.columns:
    # Convertir cadenas de geometría a objetos geométricos usando `loads` de Shapely
    grid['geometry'] = grid['geometry'].apply(lambda x: loads(x) if isinstance(x, str) else x)

    # Ahora que todos los valores en 'geometry' son objetos geométricos, crea el GeoDataFrame
    grid = gpd.GeoDataFrame(grid, geometry='geometry')
else:
    print("La columna 'geometry' no existe en 'grid'.")

In [ ]:
# Paso 1: Eliminar espacios al inicio en 'name'
df1['name'] = df1['name'].str.lstrip()

# Paso 2: Quitar acentos
def quitar_acentos(texto):
    if isinstance(texto, str):
        return unidecode.unidecode(texto)
    return texto

for columna in directorio_full.select_dtypes(include=['object']).columns:
    df1[columna] = df1[columna].apply(quitar_acentos)

In [ ]:
# Convertir df1 en GeoDataFrame
gdf1 = gpd.GeoDataFrame(df1, geometry=gpd.points_from_xy(df1.Longitude, df1.Latitude))

# Aquí estoy asumiendo que tus datos están en WGS84 (EPSG:4326)
gdf1.set_crs(epsg=4326, inplace=True)
grid.set_crs(epsg=4326, inplace=True)

# Realizar la operación sjoin
gdf1_sjoined = gpd.sjoin(gdf1, grid, how="left", op="within")

# Asignar los 'ID_Poligono' a df1 desde el resultado del sjoin
df1["ID_Poligono"] = gdf1_sjoined["ID_Poligono"]

# Verificar los resultados
print(df1.head())

In [ ]:
ruta_archivo12 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv'

# Cargar el archivo .xls en un DataFrame
df_ciudades = pd.read_csv(ruta_archivo12)

In [ ]:
# Primero, nos aseguramos de que df_ciudades tenga valores únicos de 'ID_Poligono'
df_ciudades_unicos = df_ciudades.drop_duplicates(subset='ID_Poligono', keep='first')

# Ahora realizamos el merge, esta vez debería evitar la duplicación de filas en df1
df_merged = df1.merge(df_ciudades_unicos[['ID_Poligono', 'estado']], on='ID_Poligono', how='left')

# Renombramos las columnas si es necesario y eliminamos las adicionales que no necesitamos
# Este paso depende de si 'merge' crea nuevas columnas o no
df_merged.rename(columns={'estado_y': 'estado'}, inplace=True)
if 'estado_x' in df_merged.columns:
    df_merged.drop('estado_x', axis=1, inplace=True)

# Actualizamos df1 con los datos fusionados
df1 = df_merged

# Verificamos los resultados
print(df1[['ID_Poligono', 'estado']].head())


In [ ]:
# Asumiendo que df1 y df_ciudades ya están cargados

# Primero, nos aseguramos de que df_ciudades tenga valores únicos de 'ID_Poligono'
df_ciudades_unicos = df_ciudades.drop_duplicates(subset='ID_Poligono', keep='first')

# Ahora realizamos el merge, esta vez debería evitar la duplicación de filas en df1
df_merged = df1.merge(df_ciudades_unicos[['ID_Poligono', 'ciudad']], on='ID_Poligono', how='left')

# Renombramos las columnas si es necesario y eliminamos las adicionales que no necesitamos
# Este paso depende de si 'merge' crea nuevas columnas o no
df_merged.rename(columns={'ciudad_y': 'ciudad'}, inplace=True)
if 'estado_x' in df_merged.columns:
    df_merged.drop('ciudad_x', axis=1, inplace=True)

# Actualizamos df1 con los datos fusionados
df1 = df_merged

# Verificamos los resultados
print(df1[['ID_Poligono', 'ciudad']].head())


In [ ]:
df1.head(60)

In [ ]:
df1 = df1.rename(columns={
    'descripcion': 'subtipo',
    'Latitude': 'latitud',
    'Longitude': 'longitud'
})


In [ ]:
df1 = df1.drop(columns=['coordenadas'])


In [ ]:
df1 = df1[['name', 'ciudad', 'estado', 'tipo', 'subtipo', 'latitud', 'longitud', 'ID_Poligono', 'direccion']]


In [ ]:
# Guardar el DataFrame resultante en un archivo .xlsx
ruta_archivo = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/SC/instituciones_sc_1.csv'  # Cambia esto por tu ruta y nombre de archivo real

df1.to_csv(ruta_archivo, index=False)

## actualizando grid

In [6]:
sc = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/1. Mapas/Metodo_1/SC/instituciones_sc_1.csv')

In [7]:
sc

,name,ciudad,estado,tipo,subtipo,latitud,longitud,ID_Poligono,direccion
0,Angeles Sin Fronteras,"Mexicali, Baja California",Baja California,OSC / ONG,Centros de Asistencia Social,32.662937,-115.494983,PG132,"Av. Reforma 244, Altos Zona Centro C.P.21100"
1,Centro de Apoyo al Trabajor Migrante,"Mexicali, Baja California",Baja California,OSC / ONG,Centros de Asistencia Social,32.660848,-115.479494,PG132,"Jabonera 6, Col. Nueva C.P. 21100"
2,Centro de Apoyo al Trabajador Migrante: Comite...,"Mexicali, Baja California",Baja California,OSC / ONG,Centros de Asistencia Social,32.660305,-115.459726,PG132,"Calle Jabonera #6, Colonia Nueva, Mexicali, Ba..."
3,Comedor Cobina A.C.,"Mexicali, Baja California",Baja California,OSC / ONG,Centros de Asistencia Social,32.660301,-115.489875,PG132,"Avenida Lerdo 416, Col. Centro C.P. 21200"
4,"Comedor Cobina, A.C.","Mexicali, Baja California",Baja California,OSC / ONG,Centros de Asistencia Social,32.660172,-115.492330,PG132,"Av. Lerdo 416, Col. Centro, C.P. 21200,Mexical..."
...,...,...,...,...,...,...,...,...,...
359,El Buen Samaritano,"Tapachula, Chiapas",Chiapas,OSC / ONG,Centros de Asistencia Social,14.884685,-92.254527,PG735,"Calle Iztapalapa 36, Col. Azteca C.P. 30786"
360,Albergue Jesus el Buen Pastor,"Tapachula, Chiapas",Chiapas,OSC / ONG,Centros de Asistencia Social,14.876615,-92.307405,PG735,Entronque a Raymundo Enriquez (a 500 metros ha...
361,Albergue Jesus el buen pastor,"Tapachula, Chiapas",Chiapas,OSC / ONG,Centros de Asistencia Social,14.865703,-92.313195,PG735,"Carretera a Raymundo Enrique, 500 metros hacia..."
362,Hecate Society Tapachula,"Tapachula, Chiapas",Chiapas,OSC / ONG,Centros de Asistencia Social,14.864105,-92.288559,PG735,"No disponible,Tapachula, Chiapas"


In [9]:
grid7x7 = gpd.read_file('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/2. arcgis(general)/grids/grid7x7/grid_punto7x7.shp')

In [11]:
sc = sc.drop(columns=['ID_Poligono'])

# Verificar el cambio
print(sc.head())

                                                name  \
0                              Angeles Sin Fronteras   
1               Centro de Apoyo al Trabajor Migrante   
2  Centro de Apoyo al Trabajador Migrante: Comite...   
3                                Comedor Cobina A.C.   
4                              Comedor Cobina, A.C.    

                      ciudad           estado       tipo  \
0  Mexicali, Baja California  Baja California  OSC / ONG   
1  Mexicali, Baja California  Baja California  OSC / ONG   
2  Mexicali, Baja California  Baja California  OSC / ONG   
3  Mexicali, Baja California  Baja California  OSC / ONG   
4  Mexicali, Baja California  Baja California  OSC / ONG   

                        subtipo    latitud    longitud  \
0  Centros de Asistencia Social  32.662937 -115.494983   
1  Centros de Asistencia Social  32.660848 -115.479494   
2  Centros de Asistencia Social  32.660305 -115.459726   
3  Centros de Asistencia Social  32.660301 -115.489875   
4  Centros d

In [12]:
from shapely.wkt import loads
if 'geometry' in grid7x7.columns:
    # Convertir cadenas de geometría a objetos geométricos usando `loads` de Shapely
    grid7x7['geometry'] = grid7x7['geometry'].apply(lambda x: loads(x) if isinstance(x, str) else x)

    # Ahora que todos los valores en 'geometry' son objetos geométricos, crea el GeoDataFrame
    grid7x7 = gpd.GeoDataFrame(grid7x7, geometry='geometry')
else:
    print("La columna 'geometry' no existe en 'grid'.")

/Users/pablouriarte/opt/anaconda3/lib/python3.9/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


In [15]:
# Convertir df1 en GeoDataFrame
sc1 = gpd.GeoDataFrame(sc, geometry=gpd.points_from_xy(sc.longitud, sc.latitud))

# Aquí estoy asumiendo que tus datos están en WGS84 (EPSG:4326)
sc1.set_crs(epsg=4326, inplace=True)
grid7x7.set_crs(epsg=4326, inplace=True)

# Realizar la operación sjoin
gdf1_sjoined = gpd.sjoin(sc1, grid7x7, how="left", op="within")

# Asignar los 'ID_Poligono' a df1 desde el resultado del sjoin
sc["ID_Poligono"] = gdf1_sjoined["ID_Poligon"]

# Verificar los resultados
print(sc.head())

                                                name  \
0                              Angeles Sin Fronteras   
1               Centro de Apoyo al Trabajor Migrante   
2  Centro de Apoyo al Trabajador Migrante: Comite...   
3                                Comedor Cobina A.C.   
4                              Comedor Cobina, A.C.    

                      ciudad           estado       tipo  \
0  Mexicali, Baja California  Baja California  OSC / ONG   
1  Mexicali, Baja California  Baja California  OSC / ONG   
2  Mexicali, Baja California  Baja California  OSC / ONG   
3  Mexicali, Baja California  Baja California  OSC / ONG   
4  Mexicali, Baja California  Baja California  OSC / ONG   

                        subtipo    latitud    longitud  \
0  Centros de Asistencia Social  32.662937 -115.494983   
1  Centros de Asistencia Social  32.660848 -115.479494   
2  Centros de Asistencia Social  32.660305 -115.459726   
3  Centros de Asistencia Social  32.660301 -115.489875   
4  Centros d

/Users/pablouriarte/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [16]:
ruta_nueva2 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/1. Mapas/Metodo_1/SC/instituciones_sc_1.csv'

# Guardar el DataFrame en un archivo Excel en la ruta especificada
sc.to_csv(ruta_nueva2, index=False)

print(f"El DataFrame ha sido guardado exitosamente en: {ruta_nueva2}")

El DataFrame ha sido guardado exitosamente en: /Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/1. Mapas/Metodo_1/SC/instituciones_sc_1.csv


### mujeres, niños y familias

In [ ]:
DIF = directorio_full[directorio_full['descripcion'] == 'DIF']

In [ ]:
DIF